<a href="https://colab.research.google.com/github/mong-head/machine_learning/blob/master/NN_AD1_reg.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# drive 저장 및 set up

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

from pandas import Series, DataFrame

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
assert tf.__version__ >= "2.0"

import tensorflow as tf
from tensorflow import keras

TensorFlow 2.x selected.


# * data 처리

***Load TADPOLE* dataset (csv file) from Google Drive***
---
*The Alzheimer's Disease Prediction Of Longitudinal Evolution
(https://tadpole.grand-challenge.org/)

### -Subjects: 1707 (1363 Train (80%) + 344 Test (20%))
### -Features: 72
*   2 demographic feature: MMSE, ADAS13
*   70 mean values of cortical thickness

데이터 불러오기

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

csv_file_train = '/content/gdrive/My Drive/BNCS401_Midterm_Project/Train_data_reupdated.csv'  # Set your path
train_data = pd.read_csv(csv_file_train)
train_data

# DXCHANGE: clinical label (1-CN, 2-MCI, 3-AD)

Mounted at /content/gdrive


,RID,DXCHANGE,AGE,MMSE,ADAS13,ST102TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST103TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST104TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST105TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST106TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST107TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST108TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST109TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST110TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST111TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST113TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST114TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST115TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST116TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST117TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST118TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST119TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST121TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST123TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST129TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST130TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST13TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST14TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST15TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST23TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST24TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST25TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST26TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST31TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST32TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST34TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST35TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST36TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST38TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST39TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST40TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST43TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST44TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST45TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST46TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST47TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST48TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST49TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST50TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST51TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST52TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST54TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST55TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST56TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST57TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST58TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST59TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST60TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST62TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST64TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST72TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST73TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST74TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST82TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST83TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST84TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST85TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST90TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST91TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST93TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST94TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST95TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST97TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST98TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16,ST99TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16
0,4084,1,68.4,30,10.00,2.700,2.635,2.613,2.904,2.311,1.647,2.139,2.652,2.604,2.480,3.095,2.144,2.792,2.207,2.903,2.617,4.117,2.701,,3.127,3.051,2.305,2.872,2.732,2.026,3.756,2.813,2.762,2.556,2.916,2.695,2.259,2.690,2.017,2.421,2.949,2.570,2.370,2.674,3.004,2.369,1.599,2.208,2.650,2.739,2.544,3.018,2.377,2.880,2.322,2.657,2.489,3.620,2.711,,2.593,2.792,2.660,1.993,3.734,2.390,2.817,2.471,2.990,2.667,2.490,2.523,2.254,2.171,2.862
1,2196,2,68.2,30,13.00,2.453,2.992,2.470,2.965,2.438,1.584,1.910,2.900,2.451,2.335,2.771,2.354,2.712,2.001,2.729,2.363,3.613,2.475,,3.196,3.334,2.343,2.729,2.627,1.742,3.383,2.647,2.758,2.394,2.634,2.334,2.241,2.824,1.865,2.383,2.866,2.334,2.793,2.413,2.874,2.316,1.478,1.909,2.780,2.589,2.133,3.036,2.329,2.687,2.070,2.783,2.594,3.405,2.367,,2.582,2.977,2.489,1.868,3.220,2.683,2.569,2.372,2.854,2.867,2.233,2.793,1.987,2.428,2.943
2,657,1,77.7,29,15.33,2.249,2.296,2.315,2.681,2.420,1.386,1.830,2.466,2.3

train_data.info()로 확인하니, 빠져있는 데이터도 있고(예측값인 ADAS13, cortical값 중에 1개), 실제로는 NaN값이지만 ' '로 채워진 데이터가 2개가 있었다.

' '로 채워져 있어 object인 두 cortical값
*   ST64TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16
*   ST123TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16

데이터 처리 순서
1. object 처리 : space to NaN
2. imputer : median으로 빠져있는 값들 채우기
3. feature scaling - RobustScaler : cortical, age 값의 범위를 같게 맞춘다.

** StandardScaler안쓰고 RobustScaler하는 이유 : 실제로 이게 더 나았음- outlier제거가 되기때문임

2,3을 pipeline으로 하고, 2 전에 AD_tr(cortical값들, age),y(reg/cls)로 나누고,

AD_tr만 pipeline해서 X로 만들고, y는 imputer만 한다.


1) object처리
---

In [0]:
#2개의 cortical feature가 float이 아니라 object이고, ' '로 채워져있다.
ob1 = train_data["ST64TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16"]
ob2 = train_data["ST123TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16"]
print(len(train_data[ob1.isnull()]), ob1.dtypes)
print(len(train_data[ob2.isnull()]),ob2.dtypes)

0 object
0 object


In [0]:
#그냥 ' '로 뜬다.
train_data["ST64TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16"][0]

' '

In [0]:
def object_to_float(o): #o = train_data["~~"] 형태
  for i in range(1363):
    if o[i] == ' ':
      o[i] = np.nan #nan으로 채움

In [0]:
object_to_float(ob1)
object_to_float(ob2)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [0]:
print(len(train_data[ob1.isnull()]), ob1.dtypes)
print(len(train_data[ob2.isnull()]),ob2.dtypes)

724 object
724 object


**train, test set나누기
---
AD_train, AD_test = 

In [0]:
#RID는 버림
train_data = train_data.drop("RID",axis=1)

** trian, test set 나누기
---
train set에서 model들의 train loss와 validation loss등을 보며 model을 결정하고

test set에서 마지막으로 검사해보았을 때에도 최적의 모델이라면 

그 모델을 선택하고, 마지막에는 train, test set 나누기 전의 X로 최종으로 선택했던 모델로 마지막 training시킨다.

In [0]:
from sklearn.model_selection import train_test_split

AD_train, AD_test = train_test_split(train_data,test_size=0.2,random_state= 42)

** AD_tr, y나누고, y에는 imputer
---

RID는 버린다.

In [0]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [0]:
##input 해당
X_prepipeline = train_data.drop("MMSE",axis = 1).drop("ADAS13",axis = 1).drop("DXCHANGE",axis = 1)
X_train_prepipeline = AD_train.drop("MMSE",axis = 1).drop("ADAS13",axis = 1).drop("DXCHANGE",axis = 1)
X_test_prepipeline = AD_test.drop("MMSE",axis = 1).drop("ADAS13",axis = 1).drop("DXCHANGE",axis = 1)

#classification input
X_plus_prepipeline = train_data.drop("DXCHANGE",axis = 1)
X_plus_train_prepipeline = AD_train.drop("DXCHANGE",axis = 1)
X_plus_test_prepipeline = AD_test.drop("DXCHANGE",axis = 1)

##output 해당
AD_label_reg = DataFrame({"MMSE":train_data["MMSE"].copy(),
                         "ADAS13":train_data["ADAS13"].copy()})
AD_label_reg_tr = DataFrame({"MMSE":AD_train["MMSE"].copy(),
                         "ADAS13":AD_train["ADAS13"].copy()})
AD_label_reg_test = DataFrame({"MMSE":AD_test["MMSE"].copy(),
                         "ADAS13":AD_test["ADAS13"].copy()})

imputer.fit(AD_label_reg)
temp = imputer.transform(AD_label_reg)
y_reg = pd.DataFrame(temp, columns=AD_label_reg.columns,
                             index=AD_label_reg.index)

imputer.fit(AD_label_reg_tr)
temp = imputer.transform(AD_label_reg_tr)
y_reg_train = pd.DataFrame(temp, columns=AD_label_reg_tr.columns,
                             index=AD_label_reg_tr.index)
imputer.fit(AD_label_reg_test)
temp = imputer.transform(AD_label_reg_test)
y_reg_test = pd.DataFrame(temp, columns=AD_label_reg_test.columns,
                             index=AD_label_reg_test.index)

y_mmse =  DataFrame({"MMSE":y_reg["MMSE"].copy()})
y_mmse_train = DataFrame({"MMSE":y_reg_train["MMSE"].copy()})
y_mmse_test = DataFrame({"MMSE":y_reg_test["MMSE"].copy()})

y_ada =  DataFrame({"ADAS13":y_reg["ADAS13"].copy()})
y_ada_train = DataFrame({"ADAS13":y_reg_train["ADAS13"].copy()})
y_ada_test = DataFrame({"ADAS13":y_reg_test["ADAS13"].copy()})

y_cls =  DataFrame({"DXCHANGE":train_data["DXCHANGE"]})
y_cls_train = DataFrame({"DXCHANGE":AD_train["DXCHANGE"]})
y_cls_test = DataFrame({"DXCHANGE":AD_test["DXCHANGE"]})

incomplete_rows_before = AD_label_reg_tr[AD_label_reg_tr.isnull().any(axis=1)]#.head()
incomplete_rows_after = y_reg_train[y_reg_train.isnull().any(axis=1)]
print("1. before imputer:\n", incomplete_rows_before)
print("\n2. after imputer : \n", incomplete_rows_after)

1. before imputer:
       MMSE  ADAS13
522     23     NaN
25      21     NaN
1311    18     NaN
999     20     NaN
426     25     NaN
946     29     NaN
431     22     NaN

2. after imputer : 
 Empty DataFrame
Columns: [MMSE, ADAS13]
Index: []


2) pipeline : imputer,RobustScaler
---
* imputer - 빈데이터 채우기
* RobustScaler - 값 범위 같게 만들기, Standard보다는 outlier다루는데 더 괜찮을 거 같아서 이것을 썼다.

In [0]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

In [0]:
#빈데이터 모두 보기
incomplete_rows = X_prepipeline[X_prepipeline.isnull().any(axis=1)]#.head()
print("빈데이터 개수: ",len(incomplete_rows))

빈데이터 개수:  724


In [0]:
data_pipeline = Pipeline([
                     ('imputer',SimpleImputer(strategy="median")),
                     ('rb_scaler',RobustScaler()),
])

In [0]:
X = data_pipeline.fit_transform(X_prepipeline)
X_train = data_pipeline.fit_transform(X_train_prepipeline)
X_test = data_pipeline.fit_transform(X_test_prepipeline)

X_plus = data_pipeline.fit_transform(X_plus_prepipeline)
X_plus_train = data_pipeline.fit_transform(X_plus_train_prepipeline)
X_plus_test = data_pipeline.fit_transform(X_plus_test_prepipeline)

# Tensorboard

In [0]:
root_logdir = os.path.join(os.curdir, "my_logs")

In [0]:
def get_run_logdir():
    import time
    run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [0]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

# 1 - Regression

In [0]:
X.shape[1:]

(1363, 71)

In [0]:
np.random.seed(42)
tf.random.set_seed(42)

In [0]:
model = keras.models.Sequential([
    keras.layers.Dense(10, activation="relu",input_shape=X.shape[1:]),
    #keras.layers.Dense(10, activation="relu"),
    keras.layers.Dense(2, activation="relu")
])
model.compile(loss="mean_squared_error", optimizer=keras.optimizers.SGD(lr=1e-3))
model.save("keras_reg_model.h5")

In [0]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
history = model.fit(X, y_reg.values, epochs=20, validation_split=0.1,batch_size=17)

Train on 1226 samples, validate on 137 samples
Epoch 1/20
1226/1226 [==============================] - 2s 1ms/sample - loss: 304.6438 - val_loss: 167.6507
Epoch 2/20
1226/1226 [==============================] - 0s 206us/sample - loss: 124.7686 - val_loss: 107.0095
Epoch 3/20
1226/1226 [==============================] - 0s 216us/sample - loss: 74.7101 - val_loss: 55.4585
Epoch 4/20
1226/1226 [==============================] - 0s 213us/sample - loss: 42.4635 - val_loss: 36.7639
Epoch 5/20
1226/1226 [==============================] - 0s 205us/sample - loss: 31.9869 - val_loss: 30.2729
Epoch 6/20
1226/1226 [==============================] - 0s 204us/sample - loss: 29.4919 - val_loss: 30.1825
Epoch 7/20
1226/1226 [==============================] - 0s 209us/sample - loss: 28.2917 - val_loss: 28.5548
Epoch 8/20
1226/1226 [==============================] - 0s 227us/sample - loss: 27.5876 - val_loss: 29.3262
Epoch 9/20
1226/1226 [==============================] - 0s 201us/sample - loss: 26.9587

In [0]:
model = keras.models.load_model("keras_reg_model.h5")
checkpoint_cb = keras.callbacks.ModelCheckpoint("keras_reg_model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [0]:
history = model.fit(X, y_reg.values, epochs=30, 
                    validation_split=0.1,batch_size=17,
                    callbacks=[checkpoint_cb, tensorboard_cb,early_stopping_cb])

Train on 1226 samples, validate on 137 samples
Epoch 1/30
1226/1226 [==============================] - 1s 743us/sample - loss: 304.6438 - val_loss: 167.6507
Epoch 2/30
1226/1226 [==============================] - 0s 232us/sample - loss: 124.7686 - val_loss: 107.0095
Epoch 3/30
1226/1226 [==============================] - 0s 237us/sample - loss: 74.7101 - val_loss: 55.4585
Epoch 4/30
1226/1226 [==============================] - 0s 231us/sample - loss: 42.4635 - val_loss: 36.7639
Epoch 5/30
1226/1226 [==============================] - 0s 224us/sample - loss: 31.9869 - val_loss: 30.2729
Epoch 6/30
1226/1226 [==============================] - 0s 233us/sample - loss: 29.4919 - val_loss: 30.1825
Epoch 7/30
1226/1226 [==============================] - 0s 224us/sample - loss: 28.2917 - val_loss: 28.5548
Epoch 8/30
1226/1226 [==============================] - 0s 227us/sample - loss: 27.5876 - val_loss: 29.3262
Epoch 9/30
1226/1226 [==============================] - 0s 222us/sample - loss: 26.95

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                720       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
Total params: 742
Trainable params: 742
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.predict(X)

array([[28.47143 , 10.136587],
       [28.726498, 12.577354],
       [27.317858, 12.610836],
       ...,
       [25.834381, 18.528952],
       [27.015495, 15.816853],
       [27.813656, 13.891432]], dtype=float32)

In [0]:
loss = tf.losses.Huber(delta=1.0)
def build_model():
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(10, activation="relu",input_shape=X.shape[1:])) 
    model.add(keras.layers.Dense(2,activation= "relu"))
    optimizer = keras.optimizers.SGD(lr=1e-3)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [0]:
test_reg = keras.wrappers.scikit_learn.KerasRegressor(build_fn=build_model,
                                                      epochs=100,batch_size=10)

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, random_state=42)
scores = cross_val_score(test_reg,X,y_reg.values,
                         scoring="neg_mean_squared_error",
                         cv=kfold)
keras_reg_scores = np.sqrt(-scores).mean()
print('keras_reg validation score(rmse) :', keras_reg_scores)

Train on 1090 samples
Epoch 1/100
1090/1090 [==============================] - 1s 488us/sample - loss: 335.8716
Epoch 2/100
1090/1090 [==============================] - 0s 273us/sample - loss: 81.3029
Epoch 3/100
1090/1090 [==============================] - 0s 261us/sample - loss: 40.7986
Epoch 4/100
1090/1090 [==============================] - 0s 249us/sample - loss: 31.1064
Epoch 5/100
1090/1090 [==============================] - 0s 265us/sample - loss: 28.7177
Epoch 6/100
1090/1090 [==============================] - 0s 261us/sample - loss: 27.3328
Epoch 7/100
1090/1090 [==============================] - 0s 261us/sample - loss: 26.5178
Epoch 8/100
1090/1090 [==============================] - 0s 277us/sample - loss: 25.9202
Epoch 9/100
1090/1090 [==============================] - 0s 271us/sample - loss: 25.4792
Epoch 10/100
1090/1090 [==============================] - 0s 248us/sample - loss: 25.1202
Epoch 11/100
1090/1090 [==============================] - 0s 258us/sample - loss: 24.8

hyperparameter tuning
---

In [0]:
keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

In [0]:
loss = tf.losses.Huber(delta=1.0)
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[71]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(2,activation= "relu"))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [0]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

In [0]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("keras_reg_model.h5", save_best_only=True)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                  restore_best_weights=True)

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [0]:
keras_reg.fit(X, y_reg.values, epochs=100,
              validation_split=0.1,
              callbacks=[checkpoint_cb, tensorboard_cb,early_stopping_cb])

Train on 1226 samples, validate on 137 samples
Epoch 1/100
1226/1226 [==============================] - 1s 416us/sample - loss: 212.4466 - val_loss: 121.4823
Epoch 2/100
1226/1226 [==============================] - 0s 135us/sample - loss: 69.9866 - val_loss: 47.5700
Epoch 3/100
1226/1226 [==============================] - 0s 136us/sample - loss: 37.8792 - val_loss: 37.5023
Epoch 4/100
1226/1226 [==============================] - 0s 138us/sample - loss: 30.9267 - val_loss: 32.3445
Epoch 5/100
1226/1226 [==============================] - 0s 133us/sample - loss: 29.0041 - val_loss: 30.3506
Epoch 6/100
1226/1226 [==============================] - 0s 141us/sample - loss: 27.8467 - val_loss: 29.9539
Epoch 7/100
1226/1226 [==============================] - 0s 137us/sample - loss: 26.7904 - val_loss: 28.6768
Epoch 8/100
1226/1226 [==============================] - 0s 125us/sample - loss: 26.1804 - val_loss: 31.1743
Epoch 9/100
1226/1226 [==============================] - 0s 126us/sample - loss

In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(keras_reg,X,y_reg.values,
                         scoring="neg_mean_squared_error",cv=5)
keras_reg_scores = np.sqrt(-scores).mean()
print('keras_reg validation score(rmse) :', keras_reg_scores)

Train on 1090 samples
1090/1090 [==============================] - 0s 363us/sample - loss: 244.1206
Train on 1090 samples
1090/1090 [==============================] - 1s 491us/sample - loss: 244.4997
Train on 1090 samples
1090/1090 [==============================] - 0s 360us/sample - loss: 242.4720
Train on 1091 samples
1091/1091 [==============================] - 0s 357us/sample - loss: 262.9601
Train on 1091 samples
1091/1091 [==============================] - 1s 492us/sample - loss: 240.1638
keras_reg validation score(rmse) : 11.111010090807369


In [0]:
train_prediction_reg = keras_reg.predict(X)
print("mmse max : ",train_prediction_reg[:,0].max())
print("mmse min :", train_prediction_reg[:,0].min())
print("adas13 max : ",train_prediction_reg[:,1].max())
print("adas13 min :", train_prediction_reg[:,1].min())
train_prediction_reg

mmse max :  32.84368
mmse min : 20.798416
adas13 max :  43.171127
adas13 min : 6.4930105


array([[29.244411 ,  8.6070795],
       [29.599188 , 12.83428  ],
       [27.565212 , 15.115613 ],
       ...,
       [26.164125 , 19.957102 ],
       [28.311487 , 16.22205  ],
       [27.3413   , 14.12544  ]], dtype=float32)

In [0]:
y_reg.values

array([[30.  , 10.  ],
       [30.  , 13.  ],
       [29.  , 15.33],
       ...,
       [26.  , 27.  ],
       [23.  , 29.33],
       [30.  , 12.  ]])

In [0]:
np.random.seed(42)
tf.random.set_seed(42)

In [0]:
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [0]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100),
    "learning_rate": reciprocal(3e-4, 3e-2),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X, y_reg.values, epochs=100,
                  validation_split=0.1,
                  callbacks=[tensorboard_cb,early_stopping_cb])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] learning_rate=0.001683454924600351, n_hidden=0, n_neurons=15 ....
Train on 817 samples, validate on 91 samples
Epoch 1/100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


817/817 [==============================] - 0s 498us/sample - loss: 485.0731 - val_loss: 403.5010
Epoch 2/100
817/817 [==============================] - 0s 127us/sample - loss: 402.2192 - val_loss: 348.3591
Epoch 3/100
817/817 [==============================] - 0s 120us/sample - loss: 360.8907 - val_loss: 325.2777
Epoch 4/100
817/817 [==============================] - 0s 131us/sample - loss: 339.3987 - val_loss: 315.8981
Epoch 5/100
817/817 [==============================] - 0s 127us/sample - loss: 327.3489 - val_loss: 311.6213
Epoch 6/100
817/817 [==============================] - 0s 120us/sample - loss: 319.8480 - val_loss: 309.3746
Epoch 7/100
817/817 [==============================] - 0s 124us/sample - loss: 314.6207 - val_loss: 307.1440
Epoch 8/100
817/817 [==============================] - 0s 125us/sample - loss: 310.5179 - val_loss: 305.3496
Epoch 9/100
817/817 [==============================] - 0s 132us/sample - loss: 306.9562 - val_loss: 303.5046
Epoch 10/100
817/817 [=========

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   11.0s remaining:    0.0s


818/818 [==============================] - 0s 492us/sample - loss: 503.1139 - val_loss: 438.0286
Epoch 2/100
818/818 [==============================] - 0s 123us/sample - loss: 442.9286 - val_loss: 393.9308
Epoch 3/100
818/818 [==============================] - 0s 120us/sample - loss: 410.3066 - val_loss: 372.8043
Epoch 4/100
818/818 [==============================] - 0s 125us/sample - loss: 392.3338 - val_loss: 363.0030
Epoch 5/100
818/818 [==============================] - 0s 122us/sample - loss: 381.9090 - val_loss: 358.2326
Epoch 6/100
818/818 [==============================] - 0s 126us/sample - loss: 375.2857 - val_loss: 355.6205
Epoch 7/100
818/818 [==============================] - 0s 139us/sample - loss: 370.7060 - val_loss: 353.6621
Epoch 8/100
818/818 [==============================] - 0s 124us/sample - loss: 367.0612 - val_loss: 351.6044
Epoch 9/100
818/818 [==============================] - 0s 131us/sample - loss: 363.8348 - val_loss: 349.1558
Epoch 10/100
818/818 [=========

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  3.2min finished
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


1226/1226 [==============================] - 1s 597us/sample - loss: 377.9575 - val_loss: 320.6213
Epoch 2/100
1226/1226 [==============================] - 0s 120us/sample - loss: 310.3236 - val_loss: 305.6258
Epoch 3/100
1226/1226 [==============================] - 0s 128us/sample - loss: 294.4113 - val_loss: 290.0092
Epoch 4/100
1226/1226 [==============================] - 0s 122us/sample - loss: 279.7119 - val_loss: 273.4662
Epoch 5/100
1226/1226 [==============================] - 0s 127us/sample - loss: 263.9540 - val_loss: 251.6123
Epoch 6/100
1226/1226 [==============================] - 0s 123us/sample - loss: 245.5638 - val_loss: 228.6680
Epoch 7/100
1226/1226 [==============================] - 0s 132us/sample - loss: 219.6861 - val_loss: 198.6389
Epoch 8/100
1226/1226 [==============================] - 0s 126us/sample - loss: 168.1395 - val_loss: 135.3512
Epoch 9/100
1226/1226 [==============================] - 0s 125us/sample - loss: 112.9173 - val_loss: 101.2214
Epoch 10/100


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor object at 0x7eff259116a0>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7eff70585128>,
                                        'n_hidden': [0, 1, 2, 3],
                                        'n_neurons': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10,...
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_trai

In [0]:
rnd_search_cv.best_params_

{'learning_rate': 0.006010328378268217, 'n_hidden': 0, 'n_neurons': 2}

In [0]:
rnd_search_cv.best_score_

-28.54502854938465

In [0]:
rnd_search_cv.best_estimator_

In [0]:
model_reg = rnd_search_cv.best_estimator_.model

In [0]:
run_logdir = get_run_logdir()
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [0]:
model_reg.fit(X, y_reg.values, epochs=100,
                  validation_split=0.1,
                  callbacks=[tensorboard_cb,early_stopping_cb])

Train on 1226 samples, validate on 137 samples
Epoch 1/100
1226/1226 [==============================] - 0s 127us/sample - loss: 25.5846 - val_loss: 26.7267
Epoch 2/100
1226/1226 [==============================] - 0s 123us/sample - loss: 25.4943 - val_loss: 26.2151
Epoch 3/100
1226/1226 [==============================] - 0s 126us/sample - loss: 25.5219 - val_loss: 26.4893
Epoch 4/100
1226/1226 [==============================] - 0s 121us/sample - loss: 25.4668 - val_loss: 26.8722
Epoch 5/100
1226/1226 [==============================] - 0s 122us/sample - loss: 25.4384 - val_loss: 26.1440
Epoch 6/100
1226/1226 [==============================] - 0s 133us/sample - loss: 25.4410 - val_loss: 26.2973
Epoch 7/100
1226/1226 [==============================] - 0s 129us/sample - loss: 25.4140 - val_loss: 26.0869
Epoch 8/100
1226/1226 [==============================] - 0s 125us/sample - loss: 25.4106 - val_loss: 26.2014
Epoch 9/100
1226/1226 [==============================] - 0s 116us/sample - loss: 

In [0]:
model_reg.summary()

Model: "sequential_36"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_78 (Dense)             (None, 2)                 144       
Total params: 144
Trainable params: 144
Non-trainable params: 0
_________________________________________________________________


In [0]:
model_reg.model

AttributeError: ignored

In [0]:
train_prediction_reg = model_reg.predict(X)
print("mmse max : ",train_prediction_reg[:,0].max())
print("mmse min :", train_prediction_reg[:,0].min())
print("adas13 max : ",train_prediction_reg[:,1].max())
print("adas13 min :", train_prediction_reg[:,1].min())
train_prediction_reg

In [0]:
model_reg = rnd_search_cv.best_estimator_

In [0]:
rnd_search_cv

In [0]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model_reg.model,X,y_reg.values,
                         scoring="neg_mean_squared_error",cv=5)
model_reg_scores = np.sqrt(-scores).mean()
print('keras_reg validation score(rmse) :', model_reg_scores)

# test set(제출용)
---

In [0]:
# Load test dataset
csv_file_test = '/content/gdrive/My Drive/BNCS401_Midterm_Project/Test_data_ageupdated.csv'  # Set your path
test_data = pd.read_csv(csv_file_test)
test_data

In [0]:
test_data = test_data.drop("RID",axis=1)

In [0]:
incomplete_rows = test_data[test_data.isnull().any(axis=1)]#.head()
incomplete_rows

In [0]:
def object_to_float(o): #o = train_data["~~"] 형태
  for i in range(343):
    if o[i] == ' ':
      o[i] = np.nan #nan으로 채움

In [0]:
object_to_float(test_data["ST64TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16"])

In [0]:
object_to_float(test_data["ST123TA_UCSFFSX_11_02_15_UCSFFSX51_08_01_16"])

In [0]:
X_ = data_pipeline.fit_transform(test_data)

In [0]:
X_.shape

In [0]:
reg_prediction = model_reg.predict(X_)
reg_prediction

In [0]:
prediction.to_csv('/content/gdrive/My Drive/Colab Notebooks/prediction_NN.csv',encoding='utf-8')